In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import requests
import re
from datetime import date
from dateutil.rrule import rrule, DAILY
from bs4 import BeautifulSoup
from time import sleep
from tqdm import tqdm
import random
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import NoSuchElementException


In [53]:
#получаем ссылки на списки новостей по дате
start_date = date(2023, 1, 1)
end_date = date(2023, 10, 24)
dates = []
for d in rrule(DAILY, dtstart=start_date, until=end_date):
    dates.append('https://ria.ru/economy/' + d.strftime("%Y%m%d"))

In [54]:
dates

['https://ria.ru/economy/20230101',
 'https://ria.ru/economy/20230102',
 'https://ria.ru/economy/20230103',
 'https://ria.ru/economy/20230104',
 'https://ria.ru/economy/20230105',
 'https://ria.ru/economy/20230106',
 'https://ria.ru/economy/20230107',
 'https://ria.ru/economy/20230108',
 'https://ria.ru/economy/20230109',
 'https://ria.ru/economy/20230110',
 'https://ria.ru/economy/20230111',
 'https://ria.ru/economy/20230112',
 'https://ria.ru/economy/20230113',
 'https://ria.ru/economy/20230114',
 'https://ria.ru/economy/20230115',
 'https://ria.ru/economy/20230116',
 'https://ria.ru/economy/20230117',
 'https://ria.ru/economy/20230118',
 'https://ria.ru/economy/20230119',
 'https://ria.ru/economy/20230120',
 'https://ria.ru/economy/20230121',
 'https://ria.ru/economy/20230122',
 'https://ria.ru/economy/20230123',
 'https://ria.ru/economy/20230124',
 'https://ria.ru/economy/20230125',
 'https://ria.ru/economy/20230126',
 'https://ria.ru/economy/20230127',
 'https://ria.ru/economy/202

In [55]:
#получаем список новостей по дню, итерируясь по страницам
import math
urls = []
for n in tqdm(dates):
  url = n
  page = requests.get(url)
  soup = BeautifulSoup(page.text, 'html')
  materials = int(soup.find_all('div', {"class":"rubric-count m-active"})[0].text.split(' ')[0])
  pages = math.ceil(materials / 20)
  date = url[-8:]
  for i in range(1, pages + 1):
    page = requests.get(url + '?page={}'.format(i))
    soup = BeautifulSoup(page.text, 'html')
    urls.extend([link.get('href')
          for link in soup.find_all('a', {"class": "list-item__title color-font-hover-only"})
          if date in link.get('href')])

100%|██████████| 297/297 [30:13<00:00,  6.10s/it]


In [56]:
urls

['https://ria.ru/20230101/golovchenko-1842775915.html',
 'https://ria.ru/20230101/zameschenie-1842775318.html',
 'https://ria.ru/20230101/tovarooborot-1842771998.html',
 'https://ria.ru/20230101/mezhpravkomissiya-1842763770.html',
 'https://ria.ru/20230101/udobreniya-1842762214.html',
 'https://ria.ru/20230101/braziliya-1842759868.html',
 'https://ria.ru/20230101/gaz-1842756756.html',
 'https://ria.ru/20230101/spg-1842753642.html',
 'https://ria.ru/20230101/tseny-1842749144.html',
 'https://radiosputnik.ria.ru/20230101/gruziya-1842739366.html',
 'https://ria.ru/20230101/moldaviya-1842719641.html',
 'https://ria.ru/20230101/avtomobili-1842718000.html',
 'https://ria.ru/20230101/zerno-1842717850.html',
 'https://ria.ru/20230101/aes-1842715341.html',
 'https://ria.ru/20230101/megaproekty-1842711078.html',
 'https://ria.ru/20230101/sanktsii-1842707489.html',
 'https://ria.ru/20230101/gazprom-1842704244.html',
 'https://ria.ru/20230101/gretsiya-1842693851.html',
 'https://ria.ru/20230101/na

In [57]:
#текст новости
def get_text(driver):
  paragraphs = driver.find_elements(By.CLASS_NAME, "article__text")
  full_text = '\n'.join([p.text for p in paragraphs])
  return full_text

In [58]:
#тэги новости, если они есть
def get_tags(driver):
  try:
    tags = driver.find_elements(By.CLASS_NAME, "article__tags-item")
    return [t.text for t in tags]
  except NoSuchElementException:
    return None

In [59]:
#время и дата новости
def get_time(driver):
  time = driver.find_element(By.CLASS_NAME, "article__info-date")
  return time.text

In [65]:
#заголовок
def get_header(driver):
  header = driver.find_element(By.CLASS_NAME, "article__title")
  return header.text

In [69]:
#раздел
def get_category(driver):
  try:
    h1 = driver.find_element(By.CLASS_NAME, 'article__supertag-header-title')
    return h1.text
  except NoSuchElementException:
    return "Экономика"

In [70]:
def get_line(url0, driver):
  driver.get(url0)
  website = 'РИА'
  section = get_category(driver)
  header = get_header(driver)
  body = get_text(driver)
  date = get_time(driver)
  tags = get_tags(driver)
  return website, section, url0, header, body, date, tags

In [ ]:
dataset = []
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome(chrome_options)
for url in tqdm(urls):
  dataset.append(get_line(url, driver))
driver.close()

In [74]:
df = pd.DataFrame(dataset)
df.columns = ['website', 'section', 'url', 'header', 'body', 'date', 'tags']

In [76]:
df.to_csv('ria.csv')